In [1]:
import torch as torch
import sys 
import cv2
import numpy as np
import pickle as pkl
from scipy.stats import norm, gamma
from scipy.interpolate import interp1d
from matplotlib import pyplot as plt
sys.path.insert(0, '/Users/evanpan/Documents/GitHub/EvansToolBox/Utils')
sys.path.insert(0, '/Users/evanpan/Documents/GitHub/Gaze_project')
sys.path.insert(0, "C:/Users/evansamaa/Documents/GitHub/EvansToolBox/Utils")
sys.path.insert(0, "C:/Users/evansamaa/Documents/GitHub/Gaze_project")
sys.path.insert(0, "C:/Users/evan1/Documents/GitHub/EvansToolBox/Utils")
sys.path.insert(0, "C:/Users/evan1/Documents/GitHub/Gaze_project")
from Geometry_Util import rotation_angles_frome_positions, rotation_axis_angle_from_vector, directions_from_rotation_angles

In [2]:
%load_ext autoreload
%autoreload 1

%aimport Signal_processing_utils
%aimport Geometry_Util

# Input Block

In [3]:
# input video
input_video_path = "F:/MASC/JALI_gaze/for_non_conversational/video_360/office_stanford.mp4"
output_path = "F:/MASC/JALI_gaze/for_non_conversational/video_360/office_stanford_non_conversational.pkl"
# models
model_path = "C:/Users/evansamaa/Documents/GitHub/saliency-driven-gaze/Unreal Project/SaccadicModel/msinet_scripted_256.pt"
bias_map_path = "C:/Users/evansamaa/Documents/GitHub/saliency-driven-gaze/Unreal Project/SaccadicModel/saccadic_bias.png"

In [74]:
class Salience_map_model:
    def __init__(self) -> None:
        self._module  = torch.jit.load(model_path, torch.device("cpu"))
    def get_fixation(self, image, bias_map, salient_weight):
        uv_coordinates = [0, 0]
        # Create cv::Mat from raw input image
        cv_image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB).astype(np.float32) / 255.0
        cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
        # Show the image in the window
        cv2.imshow("Image", cv_image)
        input_tensor = torch.from_numpy(cv_image)
        input_tensor = input_tensor.permute((2, 0, 1)).contiguous().unsqueeze(0)

        # Normalize channels
        input_tensor[:, 0, :, :] = (input_tensor[:, 0, :, :] - 0.485) / 0.229
        input_tensor[:, 1, :, :] = (input_tensor[:, 1, :, :] - 0.456) / 0.224
        input_tensor[:, 2, :, :] = (input_tensor[:, 2, :, :] - 0.406) / 0.225
        input_tensor = input_tensor.to(torch.device('cpu'))

        # Pass Data Through Neural Net to Get Output
        result = self._module.forward([input_tensor]).to('cpu')
        salmap_tensor = result.squeeze(0).detach()

        # Saliency tensor normalization
        salmap_tensor -= torch.min(salmap_tensor)
        salmap_tensor /= (1e-7 + torch.max(salmap_tensor))

        # Convert to cv::Mat
        salmap_tensor = salmap_tensor.permute((1, 2, 0)).to('cpu')
        cv_salmap = salmap_tensor.numpy()

        # Apply salient weight
        cv_salmap = (cv_salmap * salient_weight) + (1.0 - salient_weight)

        # Get bias map
        cv_bias = np.array(bias_map[:, :, 0], dtype=np.float32) / 255.0

        # Saliency map to probability distribution
        sum_salmap = np.sum(cv_salmap)
        sum_bias = np.sum(cv_bias)

        # Normalize distributions
        cv_salmap /= sum_salmap
        cv_bias /= sum_bias

        # Create the fixation probability map
        fixation_proba = cv_bias * cv_salmap

        # Initialize the random generator for sampling
        np.random.seed()
        gen = np.random.default_rng()

        # Cast the Mat into a float vector
        fix_proba = fixation_proba.flatten()

        # Creates the distribution
        fixation_distribution = np.random.choice(len(fix_proba), size=len(fix_proba), p=fix_proba)

        # Sample N pixels in the 256*256 vector
        nb_samples = 20
        samples = fixation_distribution[:nb_samples]
        probas = fix_proba[samples]

        # Keep the max proba
        maxProbaIndex = np.argmax(probas)
        fix_sample = samples[maxProbaIndex]

        # Height
        uv_coordinates[0] = fix_sample % self._image_size
        # Width
        uv_coordinates[1] = int(fix_sample / self._image_size)

        return uv_coordinates

# Load models and inputs videos

In [82]:
model = Salience_map_model()
image = cv2.imread(bias_map_path)
T_h_sampler = norm(0.15, 0.1)
X_h_sampler = norm(40, 5)
Y_h_sampler = norm(15, 2)
fixation_duration_sampler = gamma(1.2394, 0.1880)
# T_h =  T_h_sampler.rvs(1)
# fixation_time = fixation_duration_sampler.rvs(1) + 0.08


# Animation System Test

## Animation Model Test

In [88]:
def mock_fixation_model(ts, arr):
    # ts = np.arange(0, arr.shape[0]) / fps
    arr_interp = interp1d(ts, arr, axis=0, kind="previous")
    def mock_fixation_point_grabber(t):
        return arr_interp(t)
    return mock_fixation_point_grabber

In [91]:
# video metadata
fps = 24.0
dt = 1/fps
inhibition_tracking = []
test_points = np.load("F:/MASC/JALI_gaze/for_non_conversational/debugging/test_look_at_points.npy")
# here is where I got the salient points
get_pt = mock_fixation_model(test_points[:, 0], test_points[:, 1:])
ts = np.arange(0, test_points[-1, 0] * fps) / fps

# simulation variables
counter = 0
next_gaze_shift = 0 # track the time for the next gaze shift ()
next_head_shift = 0 # track the time for the next gaze shift ()
current_gaze_target = np.array([0, 0])
current_gaze_pos = np.array([0, 0])
current_gaze_vel = np.array([0, 0])
current_head_target = np.array([0, 0]) 
current_head_vel = np.array([0, 0])
current_head_pos = np.array([0, 0])


gaze_start = False
head_start = False
# track outputs
gaze_points = [np.expand_dims(current_gaze_pos, axis=0)]
head_points = [np.expand_dims(current_head_pos, axis=0)]
# Loop through all frames in the video
vel = []
for counter in range(0, len(ts)):
    t = counter * dt
    # =========== For gaze shift =========== 
    fixation_point = rotation_angles_frome_positions(get_pt(t))
    if np.linalg.norm(fixation_point - current_gaze_target) >= 0.001:
    # if t > next_gaze_shift:
        # ========================= swap real saccadic model with get_pt(t) =========================
        fixation_point = rotation_angles_frome_positions(get_pt(t))
        # ========================= swap real saccadic model with get_pt(t) =========================
        # define the new gaze target
        current_gaze_target = fixation_point
        # find the next time gaze shift should happen 
        next_gaze_shift = t + fixation_duration_sampler.rvs(1)[0] + 0.08
        next_head_shift = t + T_h_sampler.rvs(1)[0]
        gaze_start = True
    # =========== For head rotation =========== 
    # find the angle between the current head angle and the gaze target
    head_vec = directions_from_rotation_angles(np.expand_dims(current_head_pos, axis=0), 1)[0]
    gaze_target = directions_from_rotation_angles(np.expand_dims(current_gaze_target, axis=0), 1)[0]
    H_angle_length = np.linalg.norm(head_vec - gaze_target)
    H_angle = np.arccos(0.5*((H_angle_length)**2) - 1) / np.pi * 180
    H_angle = np.linalg.norm(current_head_pos - current_gaze_target)
    print(t, next_head_shift, H_angle)
    if t > next_head_shift:
        # update a new head target when H_angle is sufficiently large
        X_h = X_h_sampler.rvs(1)[0]
        if H_angle > X_h:
            head_start = True
            current_head_target = current_gaze_target
            # update this so this won't get updated
            next_head_shift = next_head_shift + 10000
    Y_h = Y_h_sampler.rvs(1)[0]
    if H_angle <= Y_h:
        head_start = False
    if gaze_start and np.linalg.norm(current_gaze_target - current_head_pos) > 0.001:
        # move gaze
        current_gaze_vel = current_gaze_target - current_gaze_pos
        if np.linalg.norm(current_gaze_vel) > 0:
            current_gaze_vel = current_gaze_vel / np.linalg.norm(current_gaze_vel) * 100
    else:
        current_gaze_vel = np.zeros([2])
        gaze_start = False
    if head_start:
        # compute acceleration vector
        current_head_accel = current_head_target - current_head_pos
        # normalize the acceleration will be 30 deg/s^2
        if np.linalg.norm(current_head_accel) > 0:
            current_head_accel = current_head_accel / np.linalg.norm(current_head_accel) * 30
        # compute head velocity based on acceleration
        current_head_vel = current_head_vel + current_head_accel * dt
        # velocity is normalized to be less than 40 deg/s
        head_vel_norm = np.linalg.norm(current_head_vel)
        clipped_head_vel_norm = np.minimum(head_vel_norm, 40)
        current_head_vel = current_head_vel / head_vel_norm * clipped_head_vel_norm
    else:
        # head will decelerate in the negative direction of the velocity
        head_decelleration = (-current_head_vel)
        # the deceleration is also 30 deg/s
        if np.linalg.norm(head_decelleration) > 0:
            head_decelleration = head_decelleration / np.linalg.norm(head_decelleration) * 30
        # compute new velocity
        new_head_vel = current_head_vel + head_decelleration * dt
        if np.dot(new_head_vel, current_head_vel) < 0:
            current_head_vel = 0
        else:
            current_head_vel = new_head_vel
    new_gaze_pos = current_gaze_pos + current_gaze_vel * dt
    new_head_pos = current_head_pos + current_head_vel * dt
    # make sure the gaze do not over shoot
    if np.dot(current_gaze_target - new_gaze_pos, current_gaze_target - current_gaze_pos) < 0:
        gaze_start = False
        current_gaze_pos = current_gaze_target
    else:
        current_gaze_pos = new_gaze_pos
    # make sure head rotation do not over shoot
    
    if np.dot(current_head_target - new_head_pos, current_head_target - current_head_pos) < 0:
        head_start = False
        current_head_pos = current_head_target
    else:
        current_head_pos = new_head_pos
    # print(np.linalg.norm(current_gaze_vel))
    gaze_points.append(np.expand_dims(current_gaze_pos, axis=0))
    head_points.append(np.expand_dims(current_head_pos, axis=0))
    

0.0 0.1163648140782566 3.9277477045560114
0.041666666666666664 0.1163648140782566 3.9277477045560114
0.08333333333333333 0.10813203913613965 37.62506383877724
0.125 0.10813203913613965 37.62506383877724
0.16666666666666666 0.10813203913613965 37.62506383877724
0.20833333333333331 10000.108132039137 37.5729805054439
0.25 10000.108132039137 37.46881383877724
0.29166666666666663 10000.108132039137 37.31256383877724
0.3333333333333333 10000.108132039137 37.1042305054439
0.375 10000.108132039137 36.84381383877723
0.41666666666666663 10000.108132039137 36.53131383877724
0.4583333333333333 10000.108132039137 36.1667305054439
0.5 10000.108132039137 35.75006383877724
0.5416666666666666 10000.108132039137 35.28131383877724
0.5833333333333333 10000.108132039137 34.76048050544391
0.625 10000.108132039137 34.18756383877724
0.6666666666666666 10000.108132039137 33.56256383877724
0.7083333333333333 10000.108132039137 32.8854805054439
0.75 10000.108132039137 32.15631383877724
0.7916666666666666 10000.

## Output the Result of the animation model

In [92]:
gaze_points_arr = np.concatenate(gaze_points, axis=0)
head_points_arr = np.concatenate(head_points, axis=0)
gaze_pos_arr = directions_from_rotation_angles(gaze_points_arr, 100)
ek = []
hk = []
ts_list = ts.tolist()
for i in range(0, ts.shape[0]):
    ek.append([ts_list[i], float(gaze_pos_arr[i, 0]), float(gaze_pos_arr[i, 1]), float(gaze_pos_arr[i, 2])])
    hk.append([ts_list[i], float(head_points_arr[i, 0]), float(head_points_arr[i, 1])])
out = {"eye_frames": [ek],
        "head_frames": [hk],
        "micro_saccade": [],
        "other_neck": [[],[],[],[],[],[]],
        "envelope":[]}
with open(output_path, 'wb') as f:
    pkl.dump(out, f, protocol=2)
print(output_path)

F:/MASC/JALI_gaze/for_non_conversational/video_360/office_stanford_non_conversational.pkl


# System Test

In [ ]:
def mock_UV_traversal(angle, image):
    # here I;m 
    w, h, c = image.shape
    center_pixel = [int(w/2), int(h/2)] 

In [ ]:
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print('Error opening video file')
    
# video metadata
fps = cap.get(cv2.CAP_PROP_FPS)
dt = 1/fps
# simulation variables (animation)
counter = 0
next_gaze_shift = 0 # track the time for the next gaze shift ()
next_head_shift = 0 # track the time for the next gaze shift ()
current_gaze_target = np.array([0, 0])
current_gaze_pos = np.array([0, 0])
current_gaze_vel = np.array([0, 0])
current_head_target = np.array([0, 0]) 
current_head_vel = np.array([0, 0])
current_head_pos = np.array([0, 0])

# simulation variable (attention)
inhibition_tracking = []
gaze_start = False
head_start = False
# track outputs
gaze_points = [np.expand_dims(current_gaze_pos, axis=0)]
head_points = [np.expand_dims(current_head_pos, axis=0)]
# Loop through all frames in the video
vel = []
while cap.isOpened():
    t = counter * dt
    # =========== For gaze shift =========== 
    fixation_point = rotation_angles_frome_positions(get_pt(t))
    if np.linalg.norm(fixation_point - current_gaze_target) >= 0.001:
    # if t > next_gaze_shift:
        # ========================= swap real saccadic model with get_pt(t) =========================
        fixation_point = rotation_angles_frome_positions(get_pt(t))
        # ========================= swap real saccadic model with get_pt(t) =========================
        # define the new gaze target
        current_gaze_target = fixation_point
        # find the next time gaze shift should happen 
        next_gaze_shift = t + fixation_duration_sampler.rvs(1)[0] + 0.08
        next_head_shift = t + T_h_sampler.rvs(1)[0]
        gaze_start = True
    # =========== For head rotation =========== 
    # find the angle between the current head angle and the gaze target
    head_vec = directions_from_rotation_angles(np.expand_dims(current_head_pos, axis=0), 1)[0]
    gaze_target = directions_from_rotation_angles(np.expand_dims(current_gaze_target, axis=0), 1)[0]
    H_angle_length = np.linalg.norm(head_vec - gaze_target)
    H_angle = np.arccos(0.5*((H_angle_length)**2) - 1) / np.pi * 180
    H_angle = np.linalg.norm(current_head_pos - current_gaze_target)
    print(t, next_head_shift, H_angle)
    if t > next_head_shift:
        # update a new head target when H_angle is sufficiently large
        X_h = X_h_sampler.rvs(1)[0]
        if H_angle > X_h:
            head_start = True
            current_head_target = current_gaze_target
            # update this so this won't get updated
            next_head_shift = next_head_shift + 10000
    Y_h = Y_h_sampler.rvs(1)[0]
    if H_angle <= Y_h:
        head_start = False
    if gaze_start and np.linalg.norm(current_gaze_target - current_head_pos) > 0.001:
        # move gaze
        current_gaze_vel = current_gaze_target - current_gaze_pos
        if np.linalg.norm(current_gaze_vel) > 0:
            current_gaze_vel = current_gaze_vel / np.linalg.norm(current_gaze_vel) * 100
    else:
        current_gaze_vel = np.zeros([2])
        gaze_start = False
    if head_start:
        # compute acceleration vector
        current_head_accel = current_head_target - current_head_pos
        # normalize the acceleration will be 30 deg/s^2
        if np.linalg.norm(current_head_accel) > 0:
            current_head_accel = current_head_accel / np.linalg.norm(current_head_accel) * 30
        # compute head velocity based on acceleration
        current_head_vel = current_head_vel + current_head_accel * dt
        # velocity is normalized to be less than 40 deg/s
        head_vel_norm = np.linalg.norm(current_head_vel)
        clipped_head_vel_norm = np.minimum(head_vel_norm, 40)
        current_head_vel = current_head_vel / head_vel_norm * clipped_head_vel_norm
    else:
        # head will decelerate in the negative direction of the velocity
        head_decelleration = (-current_head_vel)
        # the deceleration is also 30 deg/s
        if np.linalg.norm(head_decelleration) > 0:
            head_decelleration = head_decelleration / np.linalg.norm(head_decelleration) * 30
        # compute new velocity
        new_head_vel = current_head_vel + head_decelleration * dt
        if np.dot(new_head_vel, current_head_vel) < 0:
            current_head_vel = 0
        else:
            current_head_vel = new_head_vel
    new_gaze_pos = current_gaze_pos + current_gaze_vel * dt
    new_head_pos = current_head_pos + current_head_vel * dt
    # make sure the gaze do not over shoot
    if np.dot(current_gaze_target - new_gaze_pos, current_gaze_target - current_gaze_pos) < 0:
        gaze_start = False
        current_gaze_pos = current_gaze_target
    else:
        current_gaze_pos = new_gaze_pos
    # make sure head rotation do not over shoot
    
    if np.dot(current_head_target - new_head_pos, current_head_target - current_head_pos) < 0:
        head_start = False
        current_head_pos = current_head_target
    else:
        current_head_pos = new_head_pos
    # print(np.linalg.norm(current_gaze_vel))
    gaze_points.append(np.expand_dims(current_gaze_pos, axis=0))
    head_points.append(np.expand_dims(current_head_pos, axis=0))
    counter = counter + 1
    

In [8]:
cap = cv2.VideoCapture(input_video_path)
# model.get_fixation()
if not cap.isOpened():
    print('Error opening video file')
    
# video metadata
fps = cap.get(cv2.CAP_PROP_FPS)
dt = 1/fps
inhibition_tracking = []



# simulation variables
counter = 0
next_gaze_shift = 0 # track the time for the next gaze shift ()
next_head_shift = 0 # track the time for the next gaze shift ()
current_gaze_target = np.array([0, 0])
current_gaze_pos = np.array([0, 0])
current_gaze_vel = np.array([0, 0])
current_head_target = np.array([0, 0]) 
current_head_vel = np.array([0, 0])
current_head_pos = np.array([0, 0])


gaze_start = False
head_start = False
# track outputs
gaze_points = [np.expand_dims(current_gaze_pos, axis=0)]
head_points = [np.expand_dims(current_head_pos, axis=0)]
# Loop through all frames in the video
while cap.isOpened():
    ret, frame = cap.read() # Read a frame from the video
    if not ret: # Check if the frame was read successfully
        break
    t = counter * dt
    # =========== For gaze shift =========== 
    if t > next_gaze_shift:
        # ========================= swap real saccadic model with get_pt(t) =========================
        fixation_point = rotation_angles_frome_positions(get_pt(t))
        # ========================= swap real saccadic model with get_pt(t) =========================
        # define the new gaze target
        current_gaze_target = fixation_point
        # find the next time gaze shift should happen 
        next_gaze_shift = t + fixation_duration_sampler.rvs(1)[0] + 0.08
        next_head_shift = next_gaze_shift + T_h_sampler.rvs(1)[0]
        gaze_start = True
    # =========== For head rotation =========== 
    # find the angle between the current head angle and the gaze target
    head_vec = directions_from_rotation_angles(np.expand_dims(current_head_pos, axis=0), 1)[0]
    gaze_target = directions_from_rotation_angles(np.expand_dims(current_gaze_target, axis=0), 1)[0]
    __, H_angle_rad = rotation_axis_angle_from_vector(head_vec, gaze_target)
    H_angle = H_angle_rad * 180/np.pi
    if t > next_head_shift:
        # update a new head target when H_angle is sufficiently large
        X_h = X_h_sampler.rvs(1)[0]
        if H_angle > X_h:
            head_start = True
            current_head_target = current_gaze_target
            # update this so this won't get updated
            next_head_shift = next_head_shift + 10000
    Y_h = Y_h_sampler.rvs(1)[0]
    if H_angle <= Y_h:
        head_start = False
    if gaze_start:
        # move gaze
        current_gaze_vel = current_gaze_target - current_head_pos
        if np.linalg.norm(current_gaze_vel) > 0:
            current_gaze_vel = current_gaze_vel / np.linalg.norm(current_gaze_vel) * 100
    else:
        current_gaze_vel = np.zeros([2])
    if head_start:
        # compute acceleration vector
        current_head_accel = current_head_target - current_head_pos
        # normalize the acceleration will be 30 deg/s^2
        if np.linalg.norm(current_head_accel) > 0:
            current_head_accel = current_head_accel / np.linalg.norm(current_head_accel) * 30
        # compute head velocity based on acceleration
        current_head_vel = current_head_vel + current_head_accel * dt
        # velocity is normalized to be less than 40 deg/s
        head_vel_norm = np.linalg.norm(current_head_vel)
        clipped_head_vel_norm = np.minimum(head_vel_norm, 40)
        current_head_vel = current_head_vel / head_vel_norm * clipped_head_vel_norm
    else:
        # head will decelerate in the negative direction of the velocity
        head_decelleration = (-current_head_vel)
        # the deceleration is also 30 deg/s
        if np.linalg.norm(head_decelleration) > 0:
            head_decelleration = head_decelleration / np.linalg.norm(head_decelleration) * 30
        # compute new velocity
        new_head_vel = current_head_vel + head_decelleration * dt
        if np.dot(new_head_vel, current_head_vel) < 0:
            current_head_vel = 0
        else:
            current_head_vel = new_head_vel
            
    new_gaze_pos = current_gaze_pos + current_gaze_vel * dt
    new_head_pos = current_head_pos + current_head_vel * dt
    # make sure the gaze do not over shoot
    if np.dot(current_gaze_target - new_gaze_pos, current_gaze_target - current_gaze_pos) < 0:
        gaze_start = False
        current_gaze_pos = current_gaze_target
    else:
        current_gaze_pos = new_gaze_pos
    # make sure head rotation do not over shoot
    if np.dot(current_head_target - new_head_pos, current_head_target - current_head_pos) < 0:
        head_start = False
        current_head_pos = current_head_target
    else:
        current_head_pos = new_head_pos
    
    gaze_points.append(np.expand_dims(current_gaze_pos, axis=0))
    head_points.append(np.expand_dims(current_head_pos, axis=0))
    
    counter += 1
    